In [ ]:
!pip install py_vollib

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for py_vollib: filename=py_vollib-1.0.1-py3-none-any.whl size=62833 sha256=226003a4eecdb871d61284d48614068b65605f049f652e6385275b02ab636f57
  Stored in directory: /root/.cache/pip/wheels/c0/5c/8f/1ed590a10a2cca3cbfa7a7caa29fb5d729b3e1f819bfda4b5e
  Created wheel for py_lets_be_rational: filename=py_lets_be_rational-1.0.1-py3-none-any.whl size=24451 sha256=08bfd6385c13cc6a70ae947ca5de325e2f22d8745477a086cd543eb8452d99fa
  Stored in directory: /root/.cache/pip/wheels/d9/20/b1/018f644bacb669d5cd6af600fe09ee0729ae5d75875a9f3043
Successfully built py_vollib py_lets_be_rational


In [ ]:
import datetime
import math
from math import log, sqrt, exp

import numpy as np
import mpmath as mp
from mpmath import quad, sqrt as mp_sqrt

import yfinance as yf
from pandas_datareader import data as pdr

from scipy.stats import norm

from py_vollib.black_scholes.implied_volatility import implied_volatility

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")


Here is the English translation in Markdown format:

---

# Part 1 & 2: Black-Scholes Pricing Implied Volatility Calculation (Volatility Implier Tool)

---

## 1. General Context and Objectives

This script implements an **enhanced Black-Scholes pricer** as well as a **robust method for calculating implied volatility** from an option's price. The objective is to compare the implied volatility calculated by this pricer to the "implied volatility smile" structure observed in the market (options quoted on Yahoo Finance).

---

## 2. Enhanced Black-Scholes Function

The **black_scholes_price_greeks** function calculates:

- **The theoretical price of an option** (call or put) using the Black-Scholes model with rates and dividends.
- **The vega**, which represents the price sensitivity with respect to volatility.

### Important Technical Points:

- **Input Safeguards**:  
  The variables $S$ (current price), $K$ (strike), $T$ (time to maturity), and $\sigma$ (volatility) are adjusted using `np.maximum` to avoid values close to zero (logarithm issues or division by zero).

- **Calculation of parameters $d_1$ and $d_2$**:  
  $$d_1 = \frac{\ln(S/K) + (r - q + 0.5\,\sigma^2)T}{\sigma\sqrt{T}}, \quad d_2 = d_1 - \sigma\sqrt{T}$$
  These quantities are essential for obtaining the cumulative probability (via the normal distribution's CDF, `norm.cdf`) and the probability density (`norm.pdf`).

- **Differentiation by Option Type**:  
  The price calculation differs depending on whether the option is a call or a put.

---

## 3. Implied Volatility Calculation

The `implied_vol_newton` function implements a robust **implied volatility search method** through:

- **Newton-Raphson Method**:  
  - $\sigma$ is initialized (here 0.3 by default).  
  - At each iteration, the difference between the theoretical price (calculated with `black_scholes_price_greeks`) and the market price is corrected by the vega, according to:
    $$\sigma_{\text{new}} = \sigma_{\text{current}} - \frac{f(\sigma)}{f'(\sigma)}$$
    where $f(\sigma) = \text{Price}_{\text{BS}} - \text{Market Price}$.

- **Bisection Method as Fallback**:  
  If the Newton method does not converge (or if vega is too low), the code switches to a dichotomous search over the interval $[0.01, 5.0]$ to find an approximate solution.

### Theoretical Foundations:

- **Black-Scholes Model**:  
  The calculation relies on the Black-Scholes model, which provides a closed-form formula for option prices, but whose equation is implicit for $\sigma$.  

- **Numerical Techniques**:  
  The Newton-Raphson method is used to invert this equation, and bisection ensures a solution even in situations where the derivative (vega) is too small.

### Newton-Raphson Method Principle:

- **Tangent Approximation**:  
  We approximate $f(x)$ by its tangent at point $x_n$ by performing a Taylor expansion:
  $$f(x) \approx f(x_n) + f'(x_n)(x - x_n)$$
  
- **Root Finding**:  
  By setting $f(x) = 0$, we solve:
  $$0 \approx f(x_n) + f'(x_n)(x - x_n)$$
  which leads to the approximation:
  $$x \approx x_n - \frac{f(x_n)}{f'(x_n)}$$

### Iterative Formula

The sequence update is performed by:
$$\boxed{x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}}$$

---

## 4. Market Data Retrieval and Preparation

Before calculating the implied volatilities, the script performs the following steps:

1. **Retrieval of risk-free rates and spot price** via yfinance for two tickers:
   - `UNDERLYING_TICKER` (e.g., "SPY") for the underlying asset.
   - `RISK_FREE_TICKER` (e.g., "^IRX") for the risk-free rate.

2. **Selection of a suitable expiration date**:
   - The list of expiration dates provided by Yahoo Finance is traversed.
   - A date is selected to be at least $\text{MATURITY\_DAYS}$ (here 30 days) in the future.  
   This ensures a maturity horizon consistent with the desired calibration.

3. **Filtering of the option chain**:
   - Only **options with a positive "lastPrice"** are kept (i.e., sufficiently liquid) and whose strikes are close to the spot price (between 80% and 120% of spot).  
   This liquidity filter avoids using market data that is too sparse or erratic.

4. **Use of historical data**:
   - For each filtered option, the script retrieves the previous day's closing price as a proxy for the option price. This is because day-of information may not always be available.

---

## 5. Calculating Implied Volatilities from Real Options

For each valid option (after filtering), the script:

- Retrieves the **market price** (historical closing price).
- Applies the `implied_vol_newton` function to calculate the implied volatility for a **call option** using the following parameters:
  - $S$: spot price.
  - $K$: option strike.
  - $T$: time to maturity (calculated in years via the number of trading days/252).
  - $r$: risk-free rate.
  - $q$: dividend yield (if applicable).

---

## 6. Final Visualization

The result of this processing chain is displayed using Plotly:

- **An interactive chart** that plots the volatility smile:
  - On the x-axis, the **strike prices** of the options.
  - On the y-axis, the **implied volatilities** calculated using the implemented method.
  - The curves (connected by a spline line) allow visualization of the IV evolution as a function of strikes, typically in a "smile" or "skew" shape.


In [ ]:
import yfinance as yf
import numpy as np
import datetime
import plotly.graph_objects as go
from scipy.stats import norm

# ---------------------------
# FONCTION BLACK-SCHOLES AMÉLIORÉE
# ---------------------------
def black_scholes_price_greeks(S, K, T, r, sigma, q=0.0, option_type="call"):
    S = np.maximum(S, 1e-12)
    K = np.maximum(K, 1e-12)
    T = np.maximum(T, 1e-6)
    sigma = np.maximum(sigma, 1e-4)

    sqrt_T = np.sqrt(T)
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*sqrt_T)
    d2 = d1 - sigma*sqrt_T

    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    pd1 = norm.pdf(d1)
    exp_qt = np.exp(-q*T)
    exp_rt = np.exp(-r*T)

    if option_type.lower() == "call":
        price = S * exp_qt * Nd1 - K * exp_rt * Nd2
        vega = S * exp_qt * sqrt_T * pd1
    else:
        price = K * exp_rt * norm.cdf(-d2) - S * exp_qt * norm.cdf(-d1)
        vega = S * exp_qt * sqrt_T * pd1

    return price, vega

# ---------------------------
# FONCTION VOLATILITÉ IMPLICITE ROBUSTE
# ---------------------------
def implied_vol_newton(option_price, S, K, T, r, q=0.0, option_type="call"):
    discount_factor = np.exp(-r * T)
    forward = S * np.exp((r - q) * T)

    if option_type == "call":
        intrinsic = max(forward - K * discount_factor, 0)
        max_price = forward * discount_factor
    else:
        intrinsic = max(K * discount_factor - forward, 0)
        max_price = K * discount_factor

    if option_price < intrinsic - 1e-8 or option_price > max_price + 1e-8:
        raise ValueError(f"Prix hors bornes d'arbitrage [K={K}]")

    sigma = 0.3  # initial guess
    tolerance = 1e-6
    max_iterations = 100

    for _ in range(max_iterations):
        price, vega = black_scholes_price_greeks(S, K, T, r, sigma, q, option_type)
        diff = price - option_price

        if abs(diff) < tolerance:
            return sigma

        if vega < 1e-12:
            sigma += 0.05 * (-1 if diff > 0 else 1)
        else:
            sigma -= diff / vega

        sigma = np.clip(sigma, 0.01, 5.0)

    lo, hi = 0.01, 5.0
    for _ in range(max_iterations):
        mid = (lo + hi) / 2
        price, _ = black_scholes_price_greeks(S, K, T, r, mid, q, option_type)
        if abs(price - option_price) < tolerance:
            return mid
        if price > option_price:
            hi = mid
        else:
            lo = mid

    return mid

# ---------------------------
# MAIN SCRIPT
# ---------------------------

UNDERLYING_TICKER = "SPY"
MATURITY_DAYS = 30  # en jours
RISK_FREE_TICKER = "^IRX"

# 1. Récupération des données
risk_free = yf.Ticker(RISK_FREE_TICKER)
r = risk_free.info.get('regularMarketPrice', 5.0) / 100

ticker = yf.Ticker(UNDERLYING_TICKER)
S = ticker.info['regularMarketPrice']
dividend_yield = ticker.info.get('dividendYield', 0.0)

# 2. Sélection de la maturité
expiration_dates = sorted(ticker.options)
if not expiration_dates:
    raise ValueError("Aucune date d'expiration n'est disponible pour ce ticker.")
today = datetime.date.today()
target_date = today + datetime.timedelta(days=MATURITY_DAYS)

selected_expiration = next(
    (d for d in expiration_dates
     if datetime.datetime.strptime(d, "%Y-%m-%d").date() >= target_date),
    expiration_dates[-1]
)

# 3. Temps à maturité
exp_date = datetime.datetime.strptime(selected_expiration, "%Y-%m-%d").date()
trading_days = np.busday_count(today.isoformat(), exp_date.isoformat())
T = trading_days / 252.0

# 4. Récupérer la chaîne d'options (on se base sur "lastPrice")
option_chain = ticker.option_chain(selected_expiration)
calls = option_chain.calls
# Ici, nous filtrons sur lastPrice, car bid/ask ne sont pas disponibles
valid_calls = calls[
    (calls['lastPrice'] > 0) &
    (calls['strike'].between(S * 0.8, S * 1.2))
]

# 5. Calcul des volatilités en récupérant les données historiques pour chaque option
strikes = []
implied_vols = []

for _, row in valid_calls.iterrows():
    try:
        contract_symbol = row['contractSymbol']
        # Créer un objet Ticker pour le contrat d'option
        opt_ticker = yf.Ticker(contract_symbol)
        # Récupérer les données historiques pour la veille
        yesterday = today - datetime.timedelta(days=1)
        hist = opt_ticker.history(start=yesterday.strftime('%Y-%m-%d'),
                                  end=today.strftime('%Y-%m-%d'),
                                  interval="1d")
        if hist.empty:
            print(f"Aucune donnée historique pour {contract_symbol}")
            continue
        # Utiliser le cours de clôture comme proxy de prix de l'option
        market_price = hist["Close"].iloc[-1]
        iv = implied_vol_newton(
            market_price,
            S,
            row['strike'],
            T,
            r,
            q=dividend_yield,
            option_type="call"
        )
        strikes.append(row['strike'])
        implied_vols.append(iv)
        print(f"Strike {row['strike']} : marché = {market_price:.4f}, IV = {iv:.4f}")
    except Exception as e:
        print(f"Strike {row['strike']} ignoré: {str(e)}")
        continue

# 6. Visualisation
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=strikes,
    y=implied_vols,
    mode='markers+lines',
    line_shape='spline',
    name='Volatilité implicite',
    hovertemplate="Strike: %{x}<br>IV: %{y:.2%}<extra></extra>"
))

fig.update_layout(
    title=f"Smile de volatilité {UNDERLYING_TICKER}<br>Maturité: {selected_expiration}",
    xaxis_title="Strike Price",
    yaxis_title="Implied Volatility",
    yaxis_tickformat=".0%",
    template="plotly_white",
    height=600,
    hoverlabel=dict(bgcolor="white", font_size=12)
)

fig.show()


ERROR:yfinance:$SPY250516C00431000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)


Aucune donnée historique pour SPY250516C00431000
Strike 432.0 : marché = 108.9600, IV = 1.5240


ERROR:yfinance:$SPY250516C00433000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)
ERROR:yfinance:$SPY250516C00434000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)


Aucune donnée historique pour SPY250516C00433000
Aucune donnée historique pour SPY250516C00434000


ERROR:yfinance:$SPY250516C00436000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)


...

Aucune donnée historique pour SPY250516C00491000
Strike 492.0 : marché = 57.3200, IV = 1.0834


ERROR:yfinance:$SPY250516C00493000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)


Aucune donnée historique pour SPY250516C00493000
Strike 494.0 : marché = 55.5100, IV = 1.0660
Strike 495.0 : marché = 54.0800, IV = 1.0480
Aucune donnée historique pour SPY250516C00496000
Strike 497.0 : marché = 52.3200, IV = 1.0312
Strike 498.0 : marché = 51.0000, IV = 1.0152


ERROR:yfinance:$SPY250516C00499000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)


Aucune donnée historique pour SPY250516C00499000
Strike 500.0 : marché = 49.1500, IV = 0.9966
Strike 501.0 : marché = 46.0000, IV = 0.9486
Strike 502.0 : marché = 49.0400, IV = 1.0080
Aucune donnée historique pour SPY250516C00503000
Strike 504.0 : marché = 47.0800, IV = 0.9872
Strike 505.0 : marché = 42.4300, IV = 0.9130
Strike 506.0 : marché = 41.8600, IV = 0.9096
Aucune donnée historique pour SPY250516C00507000
Strike 508.0 : marché = 42.0400, IV = 0.9255
Strike 509.0 : marché = 42.7300, IV = 0.9438
Strike 510.0 : marché = 38.8000, IV = 0.8817
Strike 511.0 : marché = 39.6000, IV = 0.9019
Strike 512.0 : marché = 36.8900, IV = 0.8608
Strike 513.0 : marché = 36.9600, IV = 0.8682
Strike 514.0 : marché = 35.8000, IV = 0.8540
Strike 515.0 : marché = 35.0600, IV = 0.8471
Strike 516.0 : marché = 34.7300, IV = 0.8473
Strike 517.0 : marché = 33.1300, IV = 0.8251
Strike 518.0 : marché = 32.5000, IV = 0.8199
Strike 519.0 : marché = 32.9300, IV = 0.8333
Strike 520.0 : marché = 30.9300, IV = 0.803

# Part 3

## 1. General Context and Objectives

1. The script implements an enhanced Black-Scholes pricer and a robust method for calculating implied volatility.  
2. The main objective is to calibrate an option pricing model using historical data (previous day's option prices) to compare the implied volatility smile derived from the model with the one observed in the market.

---

## 2. Retrieval and Preparation of Historical Market Data

1. The script queries Yahoo Finance to retrieve the underlying asset price and the risk-free rate.  
2. An option expiration date is selected to correspond to approximately 30 days to maturity.  
3. Only options whose strike falls within the interval $[0.8 \times S_0, 1.2 \times S_0]$ are retained.  
4. For each option, the previous day's closing price is retrieved via the `history()` method (or, by default, the value from the `lastPrice` field is used).  
5. This data (strikes, market prices, implied volatilities provided by Yahoo Finance) will serve as the basis for model calibration.

---

## 3. PDE Resolution

### 1. Construction of the Space-Time Grid

- A grid is defined for the underlying variable $S$ ranging from $0.8 \times S_0$ to $1.2 \times S_0$ (to focus on the smile's area of interest) and for time, with the initial time equal to the maturity $T$ and decreasing to 0 (i.e., backward).  
- This grid allows discretization of both space (underlying values) and time (option evolution until expiration).

### 2. Finite Difference Scheme in Backward Mode

- The method involves solving the PDE starting from the terminal condition – the known payoff at expiration (for a European call, this is $\max(S-K, 0)$) – and moving backward in time to $t=0$ where the option price is sought.  
- For each time step, the PDE is discretized to obtain a tridiagonal linear system. The system coefficients are calculated based on local volatility (which itself depends on time and $S$) and the interest rate.
- The coefficients of the lower diagonal, main diagonal, and upper diagonal represent, respectively, the impact of second-order and first-order derivatives in the equation.  
- The tridiagonal system is solved using a dedicated method (e.g., via `solve_banded` in SciPy or `tf.linalg.tridiagonal_solve` with TensorFlow) to obtain the numerical scheme solution at each time step.

### 3. Interpolation to Obtain the Theoretical Price

- Once the PDE solution has been obtained on the grid for $t=0$, the theoretical option price for the current underlying $S_0$ is found by interpolating the solution on the spatial grid.  
- This value corresponds to the call price predicted by the calibrated PDE model.

---

## 4. Parameter Calibration

### 1. Definition of the Loss Function

- The loss function is constructed to measure the mean squared error between option prices obtained from the PDE resolution and prices observed in the market (retrieved from historical data).  
- For each strike in the option chain (e.g., in the interval $[0.8 \times S_0, 1.2 \times S_0]$), the difference between the theoretical price (from the PDE solution) and the market price is calculated.  
- The average of these squared differences constitutes the loss function to be minimized.

### 2. Parameter Optimization

- The model parameters (e.g., $\sigma_0$, $\rho$, $\gamma$, $\alpha$, $\nu$) directly influence the finite difference scheme coefficients, and therefore the prices calculated by the PDE.  
- The calibration objective is to find the combination of these parameters that minimizes the error between theoretical and observed prices.  
- For this purpose, an optimization algorithm such as L-BFGS-B is used. This algorithm iteratively adjusts the parameters by evaluating the loss function at each iteration and moving in the direction that reduces the mean squared error.
- Although optimization is performed on the CPU with SciPy, the PDE calculations in the loss function leverage the GPU via TensorFlow to accelerate resolution, enabling calibration results in reduced time.

### 3. Control and Convergence

- Reasonable bounds are imposed on the parameters (e.g., $\sigma_0 \in [0.1, 0.5]$, $\rho \in [-0.1, 0.1]$, etc.) to avoid unrealistic solutions.  
- The loss function is recalculated for each proposed set of parameters, and optimization continues until the loss function is sufficiently low or a maximum number of iterations is reached.

This calibration process allows "matching" the prices calculated by the model with market prices, thus obtaining optimal parameters for the PDE pricing model. This is crucial to ensure that the implied volatility smile derived from the model is consistent with what is observed in the market.

---

## 5. Results Visualization

1. Once the model is calibrated, the script solves the PDE for each strike and thus obtains theoretical option prices.  
2. Implied volatility is derived from prices calculated by the Black-Scholes pricer (via the Newton method) for both market options and those calculated by the PDE model.  
3. Three series are plotted in the volatility smile graph:
   - Implied volatility calculated from market prices (via the Newton method).  
   - Implied volatility obtained from PDE model prices.  
   - Implied volatility provided directly by Yahoo Finance (values are converted to percentages through specific formatting).
4. The graph is interactive (built with Plotly), thus facilitating comparison between the model and the market.

---

## 6. Main Execution and Time Measurement

1. The main script sequentially executes:
   - Historical data retrieval.
   - Parameter calibration via optimization.
   - PDE resolution for each strike.
   - Generation of interactive graphs to visualize the results.
2. Total execution time is measured, which ensures that the entire process runs within a reasonable timeframe (ideally less than 5 minutes to obtain preliminary results).


In [ ]:
import numpy as np
import yfinance as yf
import datetime
from scipy.optimize import minimize
from scipy.linalg import solve_banded
from py_vollib.black_scholes.implied_volatility import implied_volatility
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import norm

# ==================================================================
# PARAMÈTRES INITIAUX ET CONFIGURATION
# ==================================================================
UNDERLYING_TICKER = "SPY"
RISK_FREE_TICKER = "^IRX"
MATURITY_DAYS = 30  # Horizon en jours
MAX_ITER = 20       # Nombre maximal d'itérations pour l'optimisation (réduit)

# ==================================================================
# FONCTION BLACK-SCHOLES AMÉLIORÉE
# ==================================================================
def black_scholes_price_greeks(S, K, T, r, sigma, q=0.0, option_type="call"):
    S = np.maximum(S, 1e-12)
    K = np.maximum(K, 1e-12)
    T = np.maximum(T, 1e-6)
    sigma = np.maximum(sigma, 1e-4)
    sqrt_T = np.sqrt(T)
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*sqrt_T)
    d2 = d1 - sigma * sqrt_T
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    pd1 = norm.pdf(d1)
    exp_qt = np.exp(-q*T)
    exp_rt = np.exp(-r*T)
    if option_type.lower() == "call":
        price = S * exp_qt * Nd1 - K * exp_rt * Nd2
        vega = S * exp_qt * sqrt_T * pd1
    else:
        price = K * exp_rt * norm.cdf(-d2) - S * exp_qt * norm.cdf(-d1)
        vega = S * exp_qt * sqrt_T * pd1
    return price, vega

# ==================================================================
# FONCTION VOLATILITÉ IMPLICITE ROBUSTE (Newton)
# ==================================================================
def implied_vol_newton(option_price, S, K, T, r, q=0.0, option_type="call"):
    discount_factor = np.exp(-r * T)
    forward = S * np.exp((r - q) * T)
    if option_type == "call":
        intrinsic = max(forward - K * discount_factor, 0)
        max_price = forward * discount_factor
    else:
        intrinsic = max(K * discount_factor - forward, 0)
        max_price = K * discount_factor
    if option_price < intrinsic - 1e-8 or option_price > max_price + 1e-8:
        raise ValueError(f"Prix hors bornes d'arbitrage [K={K}]")
    sigma = 0.3
    tolerance = 1e-6
    max_iterations = 50  # réduit
    for _ in range(max_iterations):
        price, vega = black_scholes_price_greeks(S, K, T, r, sigma, q, option_type)
        diff = price - option_price
        if abs(diff) < tolerance:
            return sigma
        if vega < 1e-12:
            sigma += 0.05 * (-1 if diff > 0 else 1)
        else:
            sigma -= diff/vega
        sigma = np.clip(sigma, 0.01, 5.0)
    lo, hi = 0.01, 5.0
    for _ in range(max_iterations):
        mid = (lo+hi)/2
        price, _ = black_scholes_price_greeks(S, K, T, r, mid, q, option_type)
        if abs(price - option_price) < tolerance:
            return mid
        if price > option_price:
            hi = mid
        else:
            lo = mid
    return mid

# ==================================================================
# RÉCUPÉRATION ET PRÉPARATION DES DONNÉES (Historique)
# ==================================================================
def fetch_market_data_historical():
    ticker = yf.Ticker(UNDERLYING_TICKER)
    risk_free = yf.Ticker(RISK_FREE_TICKER)
    spot_price = ticker.info['regularMarketPrice']
    r = risk_free.info.get('regularMarketPrice', 5.0) / 100
    exp_dates = ticker.options
    if not exp_dates:
        raise ValueError("Aucune date d'expiration n'est disponible.")
    today = datetime.date.today()
    target_date = today + datetime.timedelta(days=MATURITY_DAYS)
    selected_expiration = next(
        (d for d in exp_dates if datetime.datetime.strptime(d, "%Y-%m-%d").date() >= target_date),
        exp_dates[-1]
    )
    expiry_date = datetime.datetime.strptime(selected_expiration, "%Y-%m-%d").date()
    trading_days = np.busday_count(today.isoformat(), expiry_date.isoformat())
    T = trading_days/252.0
    chain = ticker.option_chain(selected_expiration)
    calls = chain.calls
    # Filtrer sur lastPrice (données de la veille)
    valid_calls = calls[
        (calls['lastPrice'] > 0) &
        (calls['strike'].between(spot_price * 0.8, spot_price * 1.2))
    ].sort_values('strike')
    strikes = valid_calls['strike'].values
    market_prices = []
    for _, row in valid_calls.iterrows():
        contract_symbol = row['contractSymbol']
        opt_ticker = yf.Ticker(contract_symbol)
        yesterday = today - datetime.timedelta(days=1)
        hist = opt_ticker.history(start=yesterday.strftime('%Y-%m-%d'),
                                  end=today.strftime('%Y-%m-%d'),
                                  interval="1d")
        if hist.empty:
            market_prices.append(row['lastPrice'])
        else:
            market_prices.append(hist["Close"].iloc[-1])
    market_prices = np.array(market_prices)
    return {
        'S0': spot_price,
        'r': r,
        'T': T,
        'strikes': strikes,
        'market_prices': market_prices,
        'yahoo_iv': valid_calls['impliedVolatility'].values
    }

# ==================================================================
# MODÈLE PDE AVEC CALIBRATION (Version allégée)
# ==================================================================
class PDE_Model:
    def __init__(self, market_data):
        self.data = market_data
        self._init_grid()
    def _init_grid(self):
        S0 = self.data['S0']
        # Réduire la grille pour accélérer le calcul
        self.S_min = 0.8 * S0
        self.S_max = 1.2 * S0
        self.N_S = 50      # grille spatiale plus petite
        self.N_t = 100     # grille temporelle plus petite
        self.dS = (self.S_max - self.S_min) / (self.N_S - 1)
        self.dt = self.data['T'] / self.N_t
        self.S_grid = np.linspace(self.S_min, self.S_max, self.N_S)
        self.t_grid = np.linspace(self.data['T'], 0, self.N_t + 1)
    def solve_option_pde(self, K, params):
        sigma_0, rho, gamma, alpha, nu = params
        def sigma(t, S):
            vol = (gamma * np.exp(-t) + alpha) * np.sqrt(
                max(0.01, sigma_0**2 + rho * (S - self.data['S0']) + 0.5 * nu * (S - self.data['S0'])**2)
            )
            return min(1.0, vol)
        C = np.zeros((self.N_S, self.N_t+1))
        C[:, -1] = np.maximum(self.S_grid - K, 0)
        for n in range(self.N_t, 0, -1):
            t = self.t_grid[n]
            lower = np.zeros(self.N_S)
            main = np.zeros(self.N_S)
            upper = np.zeros(self.N_S)
            b = C[:, n].copy()
            for i in range(1, self.N_S-1):
                S_val = self.S_grid[i]
                sig = sigma(t, S_val)
                a = 0.5 * self.dt * (sig**2 * S_val**2 / self.dS**2 - self.data['r'] * S_val / self.dS)
                beta = 1.0 + self.dt * (sig**2 * S_val**2 / self.dS**2 + self.data['r'])
                c = 0.5 * self.dt * (sig**2 * S_val**2 / self.dS**2 + self.data['r'] * S_val / self.dS)
                lower[i] = -a
                main[i] = beta
                upper[i] = -c
            main[0], b[0] = 1.0, 0.0
            main[-1], b[-1] = 1.0, self.S_max - K * np.exp(-self.data['r']*(self.data['T']-self.t_grid[n-1]))
            diagonals = np.vstack([upper, main, lower])
            C[:, n-1] = solve_banded((1,1), diagonals, b)
        return np.interp(self.data['S0'], self.S_grid, C[:, 0])
    def calibrate(self):
        def loss(params):
            try:
                prices = np.array([self.solve_option_pde(K, params) for K in self.data['strikes']])
                return np.mean((prices - self.data['market_prices'])**2)
            except Exception:
                return np.inf
        bounds = [
            (0.1, 0.5),    # sigma_0
            (-0.1, 0.1),   # rho
            (0.0, 1.0),    # gamma
            (0.0, 1.0),    # alpha
            (0.0, 0.01)    # nu
        ]
        res = minimize(loss,
                       x0=[0.2, -0.01, 0.5, 0.5, 0.001],
                       method="L-BFGS-B",
                       bounds=bounds,
                       options={"maxiter": MAX_ITER, "disp": True})
        return res.x

# ==================================================================
# VISUALISATION DES RÉSULTATS
# ==================================================================
def plot_results(market_data, model_prices, params):
    def calc_iv(prices):
        iv_list = []
        for p, K in zip(prices, market_data['strikes']):
            try:
                iv = implied_volatility(p, market_data['S0'], K, market_data['T'], market_data['r'], 'c')
            except Exception:
                iv = np.nan
            iv_list.append(iv)
        return iv_list
    market_iv_computed = calc_iv(market_data['market_prices'])
    model_iv = calc_iv(model_prices)
    yahoo_iv = market_data['yahoo_iv']  # On suppose les valeurs en décimal (0.25=25%)
    fig = make_subplots(rows=1, cols=2, subplot_titles=(
        f"Prix Options - Maturité {MATURITY_DAYS} J",
        f"Smile de Volatilité - Maturité {MATURITY_DAYS} J"
    ))
    fig.add_trace(go.Scatter(
        x=market_data['strikes'], y=market_data['market_prices'],
        mode='markers', name='Prix marché', marker=dict(color='royalblue', size=8)
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=market_data['strikes'], y=model_prices,
        mode='lines', name='Prix modèle', line=dict(color='firebrick', width=2)
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=market_data['strikes'], y=market_iv_computed,
        mode='markers', name='IV calculée', marker=dict(color='royalblue', size=8),
        hovertemplate="Strike: %{x}<br>IV calculée: %{y:.2%}<extra></extra>"
    ), row=1, col=2)
    fig.add_trace(go.Scatter(
        x=market_data['strikes'], y=model_iv,
        mode='lines', name='IV modèle', line=dict(color='firebrick', width=2),
        hovertemplate="Strike: %{x}<br>IV modèle: %{y:.2%}<extra></extra>"
    ), row=1, col=2)
    fig.add_trace(go.Scatter(
        x=market_data['strikes'], y=yahoo_iv,
        mode='markers+lines', name='IV Yahoo', line=dict(color='green', width=2),
        marker=dict(color='green', size=8),
        hovertemplate="Strike: %{x}<br>IV Yahoo: %{y:.2%}<extra></extra>"
    ), row=1, col=2)
    fig.update_layout(
        template="plotly_white",
        title=f"Calibration PDE (Données d'hier) - {UNDERLYING_TICKER}",
        width=1200,
        height=600,
        hovermode="x unified",
        showlegend=True
    )
    fig.update_xaxes(title_text="Strike", row=1, col=1)
    fig.update_yaxes(title_text="Prix", row=1, col=1)
    fig.update_xaxes(title_text="Strike", row=1, col=2)
    fig.update_yaxes(title_text="IV (en %)", row=1, col=2)
    params_text = (
        f"<b>Paramètres calibrés:</b><br>"
        f"σ₀ = {params[0]:.3f}<br>ρ = {params[1]:.4f}<br>"
        f"γ = {params[2]:.2f}<br>α = {params[3]:.2f}<br>ν = {params[4]:.5f}"
    )
    fig.add_annotation(
        x=0.05, y=0.95, xref="paper", yref="paper",
        text=params_text, showarrow=False,
        bgcolor="white", bordercolor="black", borderwidth=1
    )
    fig.show()

# ==================================================================
# EXÉCUTION PRINCIPALE
# ==================================================================
if __name__ == "__main__":
    market_data = fetch_market_data_historical()
    print(f"Données marché récupérées pour {len(market_data['strikes'])} strikes.")
    print("\nDébut de la calibration...")
    model = PDE_Model(market_data)
    params = model.calibrate()
    print(f"Paramètres calibrés : {params}")
    print("\nCalcul des prix PDE...")
    calibrated_prices = [model.solve_option_pde(K, params) for K in market_data['strikes']]
    print("\nGénération des graphiques...")
    plot_results(market_data, calibrated_prices, params)


ERROR:yfinance:$SPY250516C00431000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)
ERROR:yfinance:$SPY250516C00433000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)
ERROR:yfinance:$SPY250516C00434000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)
ERROR:yfinance:$SPY250516C00436000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)
ERROR:yfinance:$SPY250516C00437000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)
ERROR:yfinance:$SPY250516C00438000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)
ERROR:yfinance:$SPY250516C00439000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)
ERROR:yfinance:$SPY250516C00442000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)
ERROR:yfinance:$SPY250516C00443000: possibly delisted; no price data found  (1d 2025-04-14 -> 2025-04-15)
ERROR:yfinance:$SPY250516C00444000: possibly d

Données marché récupérées pour 211 strikes.

Début de la calibration...
Paramètres calibrés : [0.47338031 0.1        0.96449593 0.96453266 0.        ]

Calcul des prix PDE...

Génération des graphiques...


Lower strike calls are more expensive because they have a higher intrinsic value.

We can see that the model is unable to capture implied volatility, even though it is able to track the price. This suggests poor calibration.

At this stage, we have two options:

- change the objective function to be minimised
- refine the calibration mesh

# Part 4: PDE Model Comparison and Volatility Smile Visualization

---

## 1. Context and Objectives

1. This script aims to compare option prices and the implied volatility smile obtained from a PDE model calibrated to different parameter sets, with actual market data.
2. It uses historical data from Yahoo Finance and calculates implied volatility via a Black-Scholes inversion function (provided by py_vollib).
3. The final objective is to observe the impact of different PDE parameter sets (e.g., "Set A", "Set B", "Set C") on theoretical prices and on the volatility smile, and to compare these results to market IVs.

---

## 2. Creating a PDE Model Instance

1. A PDE model instance is created from a market data dictionary that includes the underlying price, risk-free rate, maturity, strikes, observed option prices, and implied volatility values provided by Yahoo Finance.
2. The PDE model instance is then ready to be used to obtain theoretical prices via its PDE resolution method.

---

## 3. Definition of Different PDE Parameter Sets

1. Several parameter sets (e.g., "Set A", "Set B", "Set C") are defined to represent different hypotheses or configurations of the PDE model.
2. Each set contains values for parameters such as initial volatility, correlation, and other constants that influence the underlying dynamics and local volatility in the PDE.

---

## 4. Calculation of Theoretical Prices and Model Implied Volatilities

1. For each parameter set, the script iterates over all available strikes and uses the PDE model's resolution method to calculate the theoretical option price (here for a European call).
2. The calculated prices are then stored in a dictionary associated with the parameter set label.
3. For each theoretical price obtained, the corresponding implied volatility is calculated by inverting the Black-Scholes formula (using the `implied_volatility` function).
4. These results are stored in another dictionary that associates each parameter set with the calculated implied volatilities (model smile).

---

## 5. Extraction of Market Implied Volatility

1. In parallel, market implied volatility is extracted from actual (historical) option prices in the market data dictionary.
2. This extraction is done by applying the Black-Scholes inversion function to market prices and other parameters, which yields a market IV vector for each strike.

---

## 6. Visualization with Plotly

1. Two interactive charts are created in a single figure:
   - The first chart compares market option prices (displayed as points) to theoretical prices calculated by the PDE model for different parameter sets (displayed as curves).
   - The second chart presents the implied volatility smile with three series:
     - IVs derived from actual option prices (market),
     - IVs calculated from the PDE model's theoretical prices,
     - IVs directly provided by Yahoo Finance.
2. Each series is associated with a distinct color, and the hover format (e.g., `%{y:.2%}`) ensures that IV values are displayed as percentages.
3. The chart is configured to allow clear and interactive comparison between market data and model results, thus facilitating visual analysis of the PDE model's performance with different parameter sets.

---

## 7. Execution Context

1. The entire process allows observation of how different parameter sets influence theoretical prices and the implied volatility smile.
2. The interactive visualization helps compare these results against market data, thus verifying the model's ability to faithfully reproduce the evolution of prices and implied volatility.


In [ ]:


# Supposons que les fonctions et variables suivantes existent déjà d'après le code initial :
# - market_data (dictionnaire contenant 'S0', 'r', 'T', 'strikes', 'market_prices', 'yahoo_iv')
# - La classe PDE_Model et sa méthode solve_option_pde(K, params)
# - La fonction implied_vol_newton (ou ici nous utilisons la fonction implied_volatility de py_vollib)

# Création d'une instance du modèle PDE à partir des données marché
pde_model = PDE_Model(market_data)

# Définir différents ensembles de paramètres pour le modèle PDE (à titre d'exemple)
param_sets = {
    "Set A": [0.2, -0.01, 0.5, 0.5, 0.001],
    "Set B": [0.25,  0.00, 0.5, 0.5, 0.001],
    "Set C": [0.2,   0.01, 0.45, 0.5, 0.001]
}

# Pour chaque ensemble, on va calculer :
# - Les prix d'options théoriques pour chaque strike
# - La volatilité implicite correspondante (en partant des prix du modèle, option call ici)
strikes = market_data['strikes']
model_prices_dict = {}
model_iv_dict = {}

for label, params in param_sets.items():
    prices = []
    for K in strikes:
        price = pde_model.solve_option_pde(K, params)
        prices.append(price)
    prices = np.array(prices)
    model_prices_dict[label] = prices

    # Calcul des IV pour chaque strike selon le modèle
    iv_list = []
    for p, K in zip(prices, strikes):
        try:
            iv = implied_volatility(p, market_data['S0'], K, market_data['T'], market_data['r'], 'c')
        except Exception:
            iv = np.nan
        iv_list.append(iv)
    model_iv_dict[label] = np.array(iv_list)

# Calcul de l'IV du marché à partir des prix marché
market_iv = []
for p, K in zip(market_data['market_prices'], strikes):
    try:
        iv = implied_volatility(p, market_data['S0'], K, market_data['T'], market_data['r'], 'c')
    except Exception:
        iv = np.nan
    market_iv.append(iv)
market_iv = np.array(market_iv)

# ==============================================================================
# TRACÉ AVEC PLOTLY : Prix d'options et smile de volatilité
# ==============================================================================
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        f"Prix d'options (Maturité {MATURITY_DAYS} J)",
        f"Smile de volatilité (IV) (Maturité {MATURITY_DAYS} J)"
    )
)

# Graphique 1 : Prix des options vs Strike
# Prix marché (points)
fig.add_trace(go.Scatter(
    x=strikes,
    y=market_data['market_prices'],
    mode='markers',
    name='Prix marché',
    marker=dict(color='black', size=8)
), row=1, col=1)

# Pour chaque ensemble de paramètres, ajout d'une courbe de prix modélisés
colors = {"Set A": "blue", "Set B": "red", "Set C": "green"}
for label, prices in model_prices_dict.items():
    fig.add_trace(go.Scatter(
        x=strikes,
        y=prices,
        mode='lines',
        name=f"Prix modèle - {label}",
        line=dict(color=colors[label], width=2)
    ), row=1, col=1)

# Graphique 2 : Smile de Volatilité
# IV calculée à partir des prix marché (points)
fig.add_trace(go.Scatter(
    x=strikes,
    y=market_iv,
    mode='markers',
    name='IV marché',
    marker=dict(color='black', size=8),
    hovertemplate="Strike: %{x}<br>IV marché: %{y:.2%}<extra></extra>"
), row=1, col=2)

# Pour chaque ensemble de paramètres, ajout d'une courbe d'IV modélisée
for label, iv_vals in model_iv_dict.items():
    fig.add_trace(go.Scatter(
        x=strikes,
        y=iv_vals,
        mode='lines',
        name=f"IV modèle - {label}",
        line=dict(color=colors[label], width=2),
        hovertemplate="Strike: %{x}<br>IV modèle: %{y:.2%}<extra></extra>"
    ), row=1, col=2)

# Mise en forme générale
fig.update_layout(
    template="plotly_white",
    title=f"Comparaison des Prix d'Options et des Volatilités Implicites selon différents paramètres PDE<br>({UNDERLYING_TICKER})",
    width=1200,
    height=600,
    hovermode="x unified",
    showlegend=True
)
fig.update_xaxes(title_text="Strike", row=1, col=1)
fig.update_yaxes(title_text="Prix", row=1, col=1)
fig.update_xaxes(title_text="Strike", row=1, col=2)
fig.update_yaxes(title_text="IV (en %)", row=1, col=2)

fig.show()


A priori le modèle semble surestimer la volatilité

## 1. Critical Analysis of Results

1.  **General Agreement and Divergences**
    *   The curves obtained from the different parameter sets (Set A, Set B, Set C) generally reproduce the trend observed in the market.
    *   However, significant discrepancies persist at the extremes of the strike range, particularly for options that are deep out-of-the-money or very close to at-the-money. This indicates that while the PDE model manages to capture the overall trend, it fails to faithfully reproduce the entire observed implied volatility surface.

2.  **Impact of Not Accounting for Dividends**
    *   The PDE model used does not account for dividends, whereas the options listed on Yahoo Finance incorporate them via an implied dividend yield or a repo rate.
    *   This omission leads to a shift in theoretical prices – particularly for calls – which can cause an overestimation of prices and distort the calculation of implied volatility.

3.  **Analysis of Parameter Sets (Set A, B, C)**
    *   **Set A**
        *   Base Volatility: $(\sigma_0 = 0.20)$
        *   Correlation: $(\rho = -0.01)$
        *   This set tends to produce lower theoretical prices and lower implied volatility for OTM calls, which can be explained by the slight negative correlation and the lower value of $(\sigma_0)$.
    *   **Set B**
        *   Base Volatility: $(\sigma_0 = 0.25)$
        *   Correlation: $(\rho = 0.0)$
        *   This set generally lies "above" the other curves, consistent with a higher initial volatility. The modeled prices increase more rapidly, especially on the OTM part of calls, which can lead to overestimating the smile when correlation is zero.
    *   **Set C**
        *   Base Volatility: $(\sigma_0 = 0.20)$
        *   Correlation: $(\rho = 0.01)$
        *   $(\gamma)$ modified to 0.45
        *   This set often positions itself between Set A and Set B in the middle region. The slight positive correlation seems to influence the shape of the curve, particularly in the ATM/OTM transition. It is sometimes observed that Set C better reproduces the central part (ATM) but diverges more at the extremes (wings).

4.  **Quality of Calibration and Loss Function**
    *   The calibration, performed by minimizing the mean squared error between the prices calculated by the PDE model and the market prices, appears sensitive to outliers for certain strikes.
    *   This sensitivity may indicate that the loss function does not sufficiently penalize deviations in certain regions or that the optimization algorithm gets stuck in a local minimum, thereby compromising the robustness of the calibration.

---

## 2. Avenues for Improvement

1.  **Incorporation of a Dividend Yield**
    *   Integrating a dividend yield or a repo rate into the PDE model would correct the bias in call pricing (and, by extension, the calculated implied volatility).
    *   Even a modest dividend yield (1–2% annualized) can bring a notable improvement in aligning theoretical prices with market data.

2.  **Refinement of Calibration**
    *   Experiment with other optimization methods (e.g., a hybrid approach combining a global search followed by local optimization) to avoid convergence towards unrepresentative local minima.
    *   Adjust the loss function, for example by applying weighting to key strikes or using an L1 norm, to reduce the impact of outliers.

3.  **Extension of the Range and Validation Across Multiple Maturities**
    *   Calibrate the model over a wider strike range and compare the results obtained for different maturities (e.g., 15, 30, 60 days) to verify the inter-maturity stability of the calibrated parameters.

4.  **Inter-Set Validation**
    *   A more systematic analysis of the different parameter sets (Set A, B, C) should be conducted to determine which offers the best overall convergence.
    *   A systematic calibration over several periods could indicate if certain sets, particularly those with higher initial volatilities (like Set B), tend to consistently overestimate prices or if specific adjustments could be made to achieve a better compromise.

---

## 3. Conclusion

1.  The results show that the PDE model successfully reproduces the general market trend for option prices and the implied volatility smile, but notable discrepancies remain.
2.  These divergences can be partly attributed to the absence of dividend accounting and the relative simplicity of the local volatility model used.
3.  To improve the quality of the calibration, it would be advisable to integrate an adjustment for dividends, refine the optimization (by adjusting the loss function and exploring other optimization algorithms), and enhance the volatility model to better capture the dynamics observed in the market.